In [1]:
import os
import numpy as np
import json
import torch
import torchvision
import numpy as np
import cv2
import random
import detectron2
import itertools
import matplotlib.pyplot as plt
from detectron2.utils.logger import setup_logger
setup_logger()
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.structures import BoxMode
from detectron2.data.datasets import register_coco_instances
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2.utils.visualizer import ColorMode

In [ ]:
def get_dataset_dicts(img_dir, json_file):
    json_file = os.path.join(img_dir, json_file)
    with open(json_file) as f:
        imgs_anns = json.load(f)

    dataset_dicts = []
    for idx, v in enumerate(imgs_anns['images']):
        record = {}
        filename = os.path.join(img_dir, v["file_name"])
        
        record["file_name"] = filename
        record["id"] = v["id"]
        record["height"] = v["height"]
        record["width"] = v["width"]
        
        annos = imgs_anns["annotations"]
        objs = []
        for anno in annos:
            if anno['image_id'] == v["id"]:
                if anno['category_id'] == 1:
                    obj = {
                        "bbox": anno['bbox'],
                        "bbox_mode": BoxMode.XYWH_ABS,
                        "segmentation": anno["segmentation"],
                        "category_id": anno['category_id'],
                        "iscrowd": 0
                    }
                else:
                    # Buat 4 point
                    xywh = anno['bbox']
                    px = [xywh[0], xywh[0]+xywh[2], xywh[0]+xywh[2], xywh[0]]
                    py = [xywh[1], xywh[1], xywh[1]+xywh[3], xywh[1]+xywh[3]]
                    poly = [(x + 0.5, y + 0.5) for x, y in zip(px, py)]
                    poly = list(itertools.chain.from_iterable(poly))
                    obj = {
                        "bbox": anno['bbox'],
                        "bbox_mode": BoxMode.XYWH_ABS,
                        "segmentation": [poly],
                        "category_id": anno['category_id'],
                        "iscrowd": 0
                    }
                objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts

In [ ]:
DATASET_PATH = "../RoboCup-Dataset/"
for d in ["train","valid"]:
    DatasetCatalog.register("robocup_" + d, lambda d=d: get_dataset_dicts(DATASET_PATH + d, d+".json"))
    MetadataCatalog.get("robocup_" + d).set(thing_classes=["ball", "field", "left_goal", "right_goal"])
robocup_metadata = MetadataCatalog.get("robocup_train")

In [ ]:
dataset_dicts = get_dataset_dicts(DATASET_PATH+"train", "train.json")
for d in random.sample(dataset_dicts, 3):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=robocup_metadata, scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    plt.imshow(vis.get_image()[:, :, ::-1])
    plt.show()

In [ ]:
cfg = get_cfg()
cfg.OUTPUT_DIR = "../weights/maskrcnn_50/"
cfg.merge_from_file("../../detectron2/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.yaml")
cfg.DATASETS.TRAIN = ("robocup_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 0
cfg.SOLVER.IMS_PER_BATCH = 16
cfg.SOLVER.BASE_LR = 0.0001
cfg.SOLVER.MAX_ITER = 2000    # 300 iterations seems good enough, but you can certainly train longer
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 16
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 4

# Cek apakah wights sudah ada
weights_filename = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
if os.path.exists(weights_filename):
    print("Load old weights")
    cfg.MODEL.WEIGHTS = weights_filename
else:
    print("Download pretrained weights")
    cfg.MODEL.WEIGHTS = "detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl"  # initialize from model zoo

trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=True)
trainer.train()

# Testing the Model

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.8 
cfg.DATASETS.TEST = ("robocup_train")
predictor = DefaultPredictor(cfg)

In [ ]:
import time
dataset_dicts = get_dataset_dicts(DATASET_PATH+"train")
for d in random.sample(dataset_dicts, 8):  
    start_time = time.time()
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1],
                   metadata=robocup_metadata, 
                   scale=0.8,
                   instance_mode=ColorMode.IMAGE_BW
    )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    plt.imshow(v.get_image()[:, :, ::-1])
    plt.show()
    end_time = time.time()
    print("FPS : ", (1/(end_time - start_time)))